In [1]:
import coremltools
from PIL import Image
import os
import re
import shutil
import json
import yaml
import csv
from collections import OrderedDict

## Copy needed images

In [2]:
android_bundle = '../../../htd-android/app/src/main/assets/Images.bundle'
ios_bundle = '../../htd/Data/Images.bundle'

shutil.rmtree(android_bundle)
os.mkdir(android_bundle)
shutil.rmtree(ios_bundle)
os.mkdir(ios_bundle)

for label in open(os.path.join(".", 'labels.txt')):
    label = label[:-1].replace(" ", "_")
    if label == "random":
        continue
    
    src_dir = os.path.join('../AllImages/', label)
    android_dst_dir = os.path.join(android_bundle, label)
    ios_dst_dir = os.path.join(ios_bundle, label)
    meta_file = os.path.join(src_dir, 'meta.yaml')
    
    os.mkdir(android_dst_dir)
    os.mkdir(ios_dst_dir)
    
    for image in os.listdir(src_dir):
        if not re.match(r'.*(\.png|\.jpg)', image, re.IGNORECASE):
            continue
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(ios_dst_dir, label + "_" + image)
        )
        shutil.copy(
            os.path.join(src_dir, image),
            os.path.join(android_dst_dir, label + "_" + image)
        )

# Load meta

In [3]:
with open('./levels.csv') as f:
    reader = csv.DictReader(f)
    tutorials = [t for t in reader if not t["name"] == "random"]
    
for tutorial in tutorials:
    label = tutorial["name"]
    dst_dir = os.path.join("../../htd/Data/Images.bundle/", label)
    tutorial.update({
        "tutorials": sorted([os.path.join(label, x) 
             for x in os.listdir(dst_dir)
             if x.endswith('.jpg')
        ])
    })

yaml.safe_dump(
    tutorials,
    open(os.path.join(android_bundle, "levels.yaml"), "w"),
    default_flow_style=False,
    allow_unicode=True
)

yaml.safe_dump(
    tutorials,
    open(os.path.join(ios_bundle, "levels.yaml"), "w"),
    default_flow_style=False,
    allow_unicode=True
)

with open('../../htd/Data/ru.lproj/Levels.strings', 'w') as f:
    f.writelines(
        ['"{0}" = "{1}";\n'.format(t["name"], t["title_ru"]) for t in tutorials]
    )
    
with open('../../htd/Data/en.lproj/Levels.strings', 'w') as f:
    f.writelines(
        ['"{0}" = "{1}";\n'.format(t["name"], t["title_en"]) for t in tutorials]
    )

In [5]:
android_bundle

'../../../htd-android/app/src/main/assets/Images.bundle'